## Putting all the files together to show the output results (no changes made)

In [7]:

MUTATION_RATE = 0.1
MUTATION_AND_CROSSOVER_RATE = 0.25
NUM_AGENTS = 50
PERFECT_SCORE = (NUM_AGENTS-1) * 2
NUM_STRATEGIES = 6
GOOD_PLAYERS = 0.3
BAD_PLAYERS = 0.3
NUM_ROWS = 4
SELECTION_FACTOR = 5



def population_in( NUM_AGENTS, good_players, bad_players, num_rows):
    # Inizializza la popolazione
    population = []

    # Calcola il numero di "good players"
    num_good_players = int(NUM_AGENTS * good_players)
    num_bad_players = int(NUM_AGENTS * bad_players)
    # Crea i "good players"
    for _ in range(num_good_players):
        strategy_weights = [np.random.uniform(0.5, 0.7), np.random.uniform(0.4, 0.6), np.random.uniform(0.1, 0.2), np.random.uniform(0, 0.2), np.random.uniform(0, 0.2), np.random.uniform(0, 0.2)]

       # np.random.shuffle(strategy_weights)
        genome_move_weights = [np.random.uniform(0, 1) for _ in range(num_rows)]
        genome = (strategy_weights, genome_move_weights)
        population.append(NimAgent(genome))
    for _ in range(num_bad_players):
        strategy_weights = [np.random.uniform(0.1, 0.15), np.random.uniform(0.2, 0.3), np.random.uniform(0.2, 0.4),np.random.uniform(0.4, 0.7), np.random.uniform(0.4, 0.7), np.random.uniform(0.7, 0.9)]
       # np.random.shuffle(strategy_weights)
        genome_move_weights = [np.random.uniform(0, 1) for _ in range(num_rows)]
        genome = (strategy_weights, genome_move_weights)
        population.append(NimAgent(genome))

    # Crea gli altri agenti
    for _ in range(NUM_AGENTS - num_good_players-num_bad_players):
        strategy_weights = [np.random.uniform(0, 1) for _ in range(NUM_STRATEGIES)]
        genome_move_weights = [np.random.uniform(0, 1) for _ in range(num_rows)]
        genome = (strategy_weights, genome_move_weights)
        population.append(NimAgent(genome))

    return population



def crossover( genome1, genome2):
    # Scegli un punto di crossover per ciascuno dei due array nel genoma
    crossover_point1 = np.random.randint(0, len(genome1[0]))
    crossover_point2 = np.random.randint(0, len(genome1[1]))

    # Crea i nuovi genomi scambiando i valori dopo il punto di crossover
    child1_genome = (np.concatenate((genome1[0][:crossover_point1], genome2[0][crossover_point1:])),
                     np.concatenate((genome1[1][:crossover_point2], genome2[1][crossover_point2:])))
    child2_genome = (np.concatenate((genome2[0][:crossover_point1], genome1[0][crossover_point1:])),
                     np.concatenate((genome2[1][:crossover_point2], genome1[1][crossover_point2:])))

    # Crea i nuovi agenti con i genomi creati


    return child1_genome, child2_genome


def mutate(genome, mutation_rate):
    # Ensure genome is within bounds after mutation
    mutated_genome = ([], [])
    for i in range(2):
        for j in range(len(genome[i])):
            if random.random() < mutation_rate:
                mutation = random.choice([-1, 1]) * random.random()
                mutated_genome[i].append(min(max(genome[i][j] + mutation, 0), 1))  # Keep genome within [0, 1]
            else:
                mutated_genome[i].append(genome[i][j])
    return mutated_genome




def evaluation(population, nim, generation):
    fitness = [0 for _ in range(len(population))]
    average_moves = [0 for _ in range(len(population))]
    nim0_percentage = [0 for _ in range(len(population))]
    # Per ogni agente nella popolazione
    for i, agent1 in enumerate(population):
        # Gioca due volte con ogni altro agente nella popolazione
        for j, agent2 in enumerate(population):
            if i != j:
                # Una volta l'agente inizia per primo
                nim.reset()
                win=0
                agent1.interested = True
                winner, num_moves1, nim0_moves = nim.play(agent1, agent2)
                if winner == 1:
                    win += 1
                average_moves[i] += num_moves1
                nim0_percentage[i] += nim0_moves

                # Una volta l'agente inizia per secondo
                nim.reset()
                winner,num_moves2, nim0_moves = nim.play(agent2, agent1)
                nim0_percentage[i] += nim0_moves
                if winner == 2:
                    win += 1

                average_moves[i] += num_moves2

                if win == 2:
                    fitness[i] += 2
                if win == 0:
                    fitness[i] -= 1




        nim0_percentage[i] = (nim0_percentage[i] / average_moves[i]) * 100
        fitness[i] = fitness[i] + generation*2
        fitness[i] -= int(average_moves[i])
    return nim0_percentage, average_moves

def reproduce(selected):
     new_population = []
     top_percent = selected[:len(selected)//SELECTION_FACTOR]
     while len(new_population)  < len(selected):
            parent1 = random.choice(top_percent)
            parent2 = random.choice(top_percent)
            if random.random() < MUTATION_RATE:
                genome1 = mutate(parent1.genome, MUTATION_RATE)
                genome2 = mutate(parent2.genome, MUTATION_RATE)
                new_population.append(NimAgent(genome1))
                new_population.append(NimAgent(genome2))
            elif random.random()> MUTATION_RATE and random.random() < MUTATION_AND_CROSSOVER_RATE:
                genome1 = mutate(parent1.genome, MUTATION_RATE)
                genome2 = mutate(parent2.genome, MUTATION_RATE)
                child1, child2 = crossover(genome1 , genome2)
                new_population.append(NimAgent(child1))
                new_population.append(NimAgent(child2))
            elif random.random() > MUTATION_AND_CROSSOVER_RATE:
                 child1, child2 = crossover(parent1.genome , parent2.genome)
                 new_population.append(NimAgent(child1))
                 new_population.append(NimAgent(child2))
     return new_population

def selection1(population, fitness):
    # Seleziona gli agenti con fitness più alta

    # Zip population and fitness
    zipped = list(zip(population, fitness))
    # Sort by fitness in descending order
    sorted_population = sorted(zipped, key=lambda x: x[1], reverse=True)
    # Return half of the sorted population
    return [x[0] for x in sorted_population[:len(population)//2]]

def selection2(population, fitness_scores):
    # Logic to select the fittest agents
    # Let's use a simple tournament selection
    selected = []
    while len(selected) < len(population) // 2:
        participant = random.sample(list(zip(population, fitness_scores)), 2)
        winner = max(participant, key=lambda x: x[1])
        selected.append(winner[0])
    return selected

def replacement(population, new_population, fitness_scores):
     half = selection1(population, fitness_scores)

     res = half + new_population
     return res



def evolution_strategy():

    population_init  = population_in(NUM_AGENTS, GOOD_PLAYERS, BAD_PLAYERS, NUM_ROWS)
    generations = 100
    best_fitness = -1
    nim = Nim(NUM_ROWS)
    bests = []
    for generation in range(generations):
        fitness_scores, average_moves = evaluation(population_init, nim, generation)
        print(f"Generation {generation} completed")
        print(f"Best Fitness: {max(fitness_scores)}")

        best_of_generation = population_init[fitness_scores.index(max(fitness_scores))]
        bests.append(best_of_generation)
        print(f"Genome: {best_of_generation.genome[0]}, \n {best_of_generation.genome[1]}")
        #print(f"Genome is : {best_agent.genome_strategy}, {best_agent.genome_row}")

        #if (max(fitness_scores) >= -PERFECT_SCORE):
            #print(f"Generation {generation} completed")
            #print(f"Best agent: {population_init[fitness_scores.index(max(fitness_scores))]}")
            #best_agent = population_init[fitness_scores.index(max(fitness_scores))]
            #print(f"Genome is : {best_agent.genome_strategy}, {best_agent.genome_row}")


        selected = selection1(population_init, fitness_scores)




        new_population = reproduce(selected)

        population_init = replacement(population_init, new_population, fitness_scores)

       # print(f"Generation {generation} completed")
        max_fitness = max(fitness_scores)

        if max_fitness > best_fitness:
            best_fitness = max_fitness
            best_agent = best_of_generation
            print(f"Best agent: {best_agent}")
            print(f"Miglior fitness: {max_fitness}")

            print(f"Genome is : {best_agent.genome_strategy}, {best_agent.genome_row}")


    print(f"Best agent: {best_agent.genome}")
    print(f"Miglior fitness: {max_fitness}")

    fitness_scores, average_moves = evaluation(bests, nim, 0)
    picked = bests[fitness_scores.index(max(fitness_scores))]
    print(f"Genome is : {picked.genome_strategy}, {picked.genome_row}")
    print(f"Fitness: {max(fitness_scores)}")
    print(f"Average moves: {average_moves[fitness_scores.index(max(fitness_scores))]}")
    return picked



In [2]:
from collections import namedtuple
import numpy as np

Nimply = namedtuple("Nimply", "row, num_objects")
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"
    def reset(self):
        self._rows = [i * 2 + 1 for i in range(len(self._rows))]
    @property
    def rows(self) -> tuple:
        return tuple(self._rows)
    #esegue una mossa di Nim
    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


    def play1(self, player1, player2):
        turn = 0
        moves1 = 0
        moves2 = 0
        while self:

            if turn == 0:
                ply = player1(self)
                #print(f"ply: player {player1} plays {ply}")
                self.nimming(ply)
                moves1 += 1
            elif turn == 1:
                ply = player2(self)
                #print(f"ply: player {player2} plays {ply}")
                self.nimming(ply)
                moves2 += 1
            turn = 1 - turn

        winner = 1 if turn == 0 else 2


        #print(f"status: Player {str(winner)} won!")
        return winner, moves1, moves2

    def play(self, player1, player2):
        turn = 0
        moves = 0

        nim0_moves = 0
        while self:

            if turn == 0:
                ply = player1.make_move(self)
                #print(f"ply: player {player1} plays {ply}")

                self.nimming(ply)
                if player1.interested:
                    moves += 1
                    if player1.nim_sum == 0:
                        nim0_moves += 1


            elif turn == 1:
                ply = player2.make_move(self)
                #print(f"ply: player {player2} plays {ply}")
                self.nimming(ply)
                if player2.interested:
                    moves += 1
                    if player1.nim_sum == 0:
                        nim0_moves += 1

            turn = 1 - turn

        winner = (1-turn) + 1


        #print(f"status: Player {str(winner)} won!")
        return winner, moves, nim0_moves






In [3]:
import random
from copy import deepcopy
import numpy as np
from collections import namedtuple


#La fitness è quante volte un agente vince contro un altro agente

#Genome è una tupla di due elementi: il primo array indica i pesi dati a ogni strategia, il secondo array indica quanti oggetti sottrarre dall i-esima riga


class NimAgent:
    def __init__(self, genoma, interested = False):
        self.genome_strategy = genoma[0]
        self.genome_row = genoma[1]
        self.strategy = self.choose_strategy()
        self.interested = interested
        self.genome= genoma
        self.nim_sum = -1;


    def __str__(self):
          if self.interested:
            return "Player sotto esame:" +  self.strategy.__name__
          else:
            return "Second Player:" +  self.strategy.__name__




    def make_move(self, nim_state) -> Nimply:

            strategy = self.choose_strategy()
            self.strategy = strategy
            ret =  strategy(nim_state)
            tmp = deepcopy(nim_state)
            tmp.nimming(ret)
            nimsum = nim_sum(tmp)
            self.nim_sum = nimsum
            return ret





            #return self.strategy(nim_state)


    def choose_strategy(self):
            # Definisci le strategie disponibili
            strategies = [self.make_genome_move, self.select_min_row, self.select_odd_row, self.gabriele, self.select_even_row, self.random_move]

            # Normalizza i valori nel genoma per farli sommare a 1
            genome_sum = sum(self.genome_strategy)
            probabilities = [value / genome_sum for value in self.genome_strategy]

            # Scegli una strategia in base alle probabilità
            strategy = random.choices(strategies, weights=probabilities, k=1)[0]

            return strategy


    def gabriele(self,state: Nim) -> Nimply:
        """Pick always the maximum possible number of the lowest row"""
        possible_moves = [(r, o) for r, c in enumerate(state._rows) for o in range(1, c + 1)]
        return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

    def random_move(self, nim_state):
            # Trova gli indici delle pile che non sono vuote
            non_empty_piles = [i for i, num in enumerate(nim_state._rows) if num > 0]

            # Scegli una pila a caso tra quelle non vuote
            pile_index = random.choice(non_empty_piles)

            # Scegli un numero a caso di oggetti da rimuovere da quella pila
            num_objects = random.randint(1, nim_state._rows[pile_index])

            return Nimply(pile_index, num_objects)
    def select_min_row(self, nim_state):
            # Trova l'indice della pila con il minor numero di oggetti
            nim_state = nim_state._rows
            min_pile_index = nim_state.index(min(i for i in nim_state if i > 0))

            # Se c'è solo una pila con più di 0 oggetti, rimuovi tutti gli oggetti da quella pila
            if nim_state.count(0) == len(nim_state) - 1:
                num_objects = nim_state[min_pile_index]
            else:
                # Altrimenti, rimuovi un solo oggetto dalla pila
                num_objects = 1

            return Nimply(min_pile_index, num_objects)



    def select_max_row(self, nim_state):
        # Trova l'indice della pila con il maggior numero di oggetti
                nim_state = nim_state._rows
                max_pile_index = nim_state.index(max(nim_state))

                # Se c'è solo una pila con più di 0 oggetti, rimuovi tutti gli oggetti da quella pila
                if nim_state.count(0) == len(nim_state) - 1:
                    num_objects = nim_state[max_pile_index]

                else:
                    # Altrimenti, rimuovi il numero massimo d i oggetti disponibili - 1
                    num_objects = nim_state[max_pile_index] - 1 if nim_state[max_pile_index] > 1 else 1

                return Nimply(max_pile_index, num_objects)

    def select_odd_row(self, nim_state):
            # Trova gli indici delle pile con un numero dispari di oggetti
            nim_state = nim_state._rows
            max_pile_index = nim_state.index(max(nim_state))

                # Se c'è solo una pila con più di 0 oggetti, rimuovi tutti gli oggetti da quella pila
            if nim_state.count(0) == len(nim_state) - 1:
                    num_objects = nim_state[max_pile_index]
                    return Nimply(max_pile_index, num_objects)

            odd_pile_indices = [i for i, num in enumerate(nim_state) if i % 2 == 1 and num > 0]

            if odd_pile_indices:
                # Se ci sono pile con un numero dispari di oggetti, scegli una di queste pile e rimuovi un oggetto
                pile_index = random.choice(odd_pile_indices)
                num_objects = 1
            else:
                # Altrimenti, scegli una pila a caso e rimuovi un elemento da quella pila
                pile_index = random.choice([i for i, num in enumerate(nim_state) if num > 0])
                num_objects = 1

            return Nimply(pile_index, num_objects)

    def select_even_row(self, nim_state):
            # Trova gli indici delle pile con un numero massimo di oggetti
            nim_state = nim_state._rows
            max_pile_index = nim_state.index(max(nim_state))

            # Se c'è solo una pila con più di 0 oggetti, rimuovi tutti gli oggetti da quella pila
            if nim_state.count(0) == len(nim_state) - 1:
                num_objects = nim_state[max_pile_index]
                return Nimply(max_pile_index, num_objects)

            even_pile_indices = [i for i, num in enumerate(nim_state) if num > 0 and i % 2 == 0]

            if even_pile_indices:
                # Se ci sono pile con un numero dispari di oggetti, scegli una di queste pile e rimuovi un oggetto
                pile_index = random.choice(even_pile_indices)
                num_objects = 1
            else:
                # Altrimenti, scegli una pila a caso e rimuovi un elemento da quella pila
                pile_index = random.choice([i for i, num in enumerate(nim_state) if num > 0])
                num_objects = 1

            return Nimply(pile_index, num_objects)

    def make_genome_move(self, nim_state):
            moves = [
                (row, min(nim_state._rows[row], max(1, int(self.genome_row[row] * nim_state._rows[row]))))
                for row in range(len(nim_state._rows)) if nim_state._rows[row] > 0
            ]
            proportional_probs = self.calculate_proportional_probs(nim_state)
            if not proportional_probs:
                return None
            #chosen_row = random.choices(range(len(proportional_probs)), proportional_probs)[0]
            chosen_move = max(moves, key=lambda x: self.genome_row[x[0]])
            nim_move = Nimply(chosen_move[0], chosen_move[1])

            return nim_move


    def optimal_move(self, nim_state):
            for row in range(len(nim_state._rows)):
                for num_objects in range(1, nim_state._rows[row] + 1):
                    tmp = deepcopy(nim_state)
                    tmp.nimming(Nimply(row, num_objects))
                    if nim_sum(tmp) == 0:
                        return Nimply(row, num_objects)
            return self.make_genome_move(nim_state)


    def calculate_proportional_probs(self, nim_state):
        total_weight = sum(self.genome_row)
        proportional_probs = [preference / total_weight for preference in self.genome_row ]
        return proportional_probs



def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
   # logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply

def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome_row = np.load("./Computational_Intelligence/lab2/genome_row.npy")
    genome_strategy = np.load("./Computational_Intelligence/lab2/genome_strategy.npy")
    genome = (genome_strategy, genome_row)
    agent = NimAgent(genome)
    return agent.make_move(state)


def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)






In [9]:
if __name__ == "__main__":
    train = True
    if train:
        best_agent = evolution_strategy()
        np.save("genome_strategy.npy", best_agent.genome_strategy)
        np.save("genome_row.npy", best_agent.genome_row)
    else:
        genome_strategy = np.load("./Computational_Intelligence/lab2/genome_strategy.npy")
        genome_row = np.load("./Computational_Intelligence/lab2/genome_row.npy")
        genome = (genome_strategy, genome_row)
        best_agent = NimAgent(genome)

    nim = Nim(NUM_ROWS)

    total_win = 0
    for i in range(10):
        #print(f"First Moves Games {i+1}")
        nim.reset()
        winner , _, _=  nim.play1(adaptive, pure_random)
        #print(f"Winner: Player{winner}")
        if winner ==1:
            total_win += 1
    print("Adaptive vs Random")
    print("Random wins: " ,10-total_win)
    print("Adaptive wins: " ,total_win)
    print("===============================")

    total_win = 0
    for i in range (10):
        #print(f"Second Moves Games {i+1}")
        nim.reset()
        winner , _, _=  nim.play1(pure_random, adaptive)
        #print(f"Winner: Player{winner}")
        if winner ==2:
            total_win += 1
    print("Random vs Adaptive")
    print("Random wins: " , 10-total_win)
    print("Adaptive wins: " ,total_win)
    print("===============================")




    print("Gabriele vs Adaptive")
    total_win = 0
    for i in range(10):
        #print(f"First Moves Games {i+1}")
        nim.reset()
        winner , _, _=  nim.play1(adaptive, gabriele)
        #print(f"Winner: Player{winner}")
        if winner ==1:
            total_win += 1

    print("Adaptive vs Gabriele")
    print("Gabriele wins: " ,10 - total_win)
    print("Adaptive wins: " , total_win)
    print("===============================")

    total_win = 0
    for i in range (10):
        #print(f"Second Moves Games {i+1}")
        nim.reset()
        winner , _, _=  nim.play1(gabriele, adaptive)
        #print(f"Winner: Player{winner}")
        if winner ==2:
            total_win += 1
    print("Gabriele vs Adaptive")
    print("Gabriele wins: ",10 - total_win)
    print("Adaptive wins: ", total_win)
    print("===============================")





    total_win = 0
    for i in range(10):
        #print(f"First Moves Games {i+1}")
        nim.reset()
        winner , _, _=  nim.play1(adaptive, optimal)
        #print(f"Winner: Player{winner}")
        if winner ==1:
            total_win += 1
    print("Adaptive vs Optimal")
    print("Optimal wins: ",10 - total_win)
    print("Adaptive wins: ", total_win)
    print("===============================")





    total_win = 0
    for i in range (10):
        #print(f"Second Moves Games {i+1}")
        nim.reset()
        winner , _, _=  nim.play1(optimal, adaptive)
        #print(f"Winner: Player{winner}")
        if winner ==2:
            total_win += 1

    print("Optimal vs Adaptive")
    print("Optimal wins: ",10 - total_win)
    print("Adaptive wins: ", total_win)
    print("===============================")


    #print(f"Total win: {total_win}")











Generation 0 completed
Best Fitness: 12.776412776412776
Genome: [0.5484528504070815, 0.5267455565140178, 0.14037640605717847, 0.03726548211098526, 0.10612564364069864, 0.09706836515156428], 
 [0.17934513344595582, 0.2942997802603182, 0.384276709922166, 0.9812553701065977]
Best agent: Player sotto esame:gabriele
Miglior fitness: 12.776412776412776
Genome is : [0.5484528504070815, 0.5267455565140178, 0.14037640605717847, 0.03726548211098526, 0.10612564364069864, 0.09706836515156428], [0.17934513344595582, 0.2942997802603182, 0.384276709922166, 0.9812553701065977]
Generation 1 completed
Best Fitness: 16.027397260273972
Genome: [0.6720526084969087, 0.4144511042227105, 0.11173046471500991, 0.009461396422560053, 0.12339747702253517, 0.10291343347726876], 
 [0, 0.7002685600656722, 0.7609249474062448, 0.9692467042883036]
Best agent: Player sotto esame:select_odd_row
Miglior fitness: 16.027397260273972
Genome is : [0.6720526084969087, 0.4144511042227105, 0.11173046471500991, 0.00946139642256005

KeyboardInterrupt: 

-> stopping here manually, just to give an exemplification of the results